In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE75132"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE75132"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE75132.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE75132.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE75132.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"TMEM45A, SERPINB5 and p16INK4A transcript levels are predictive for development of high-grade cervical lesions"
!Series_summary	"Women persistently infected with human papillomavirus (HPV) type 16 are at high risk for development of cervical intraepithelial neoplasia grade 3 or cervical cancer (CIN3+). We aimed to identify biomarkers for progression to CIN3+ in women with persistent HPV16 infection. In this prospective study, 11,088 women aged 20–29 years were enrolled during 1991-1993, and re-invited for a second visit two years later. Cervical cytology samples obtained at both visits were tested for HPV DNA by Hybrid Capture 2 (HC2), and HC2-positive samples were genotyped by INNO-LiPA. The cohort was followed for up to 19 years via a national pathology register. To identify markers for progression to CIN3+, we performed microarray analysis on RNA extracted from cervical swabs of 30 women with persistent HPV16-infection and 11 HPV-negative women

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series summary, this dataset appears to contain gene expression data
# (mentions microarray analysis and mRNA expression levels)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Analyzing sample characteristics dictionary

# For trait (cervical cancer):
# Key 1 contains category info: 0 = normal, 1 = hpv without progression, 2 = hpv with progression
# Key 3 contains disease state: none, moderate dysplasia, severe dysplasia, CIS, cancer
# We'll use Key 1 as it directly indicates progression
trait_row = 1

# For age:
# There is no age information in the sample characteristics dictionary
age_row = None

# For gender:
# This study focuses on women (from background information), but there's no explicit gender field
# All participants are women, which makes gender a constant and not useful for our study
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert trait values to binary:
    0 = normal, 1 = HPV without progression -> 0 (no cancer/high-grade lesion)
    2 = HPV with progression -> 1 (has cancer/high-grade lesion)
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        category = int(value)
        # 0, 1 = no progression to CIN3+, 2 = progression to CIN3+
        return 1 if category == 2 else 0
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Placeholder function for age conversion"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Get clinical data from the clinical_data DataFrame
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("Preview of selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{'GSM1943705': [0.0], 'GSM1943706': [0.0], 'GSM1943707': [0.0], 'GSM1943708': [0.0], 'GSM1943709': [0.0], 'GSM1943710': [0.0], 'GSM1943711': [0.0], 'GSM1943712': [1.0], 'GSM1943713': [1.0], 'GSM1943714': [1.0], 'GSM1943715': [0.0], 'GSM1943716': [0.0], 'GSM1943717': [0.0], 'GSM1943718': [0.0], 'GSM1943719': [0.0], 'GSM1943720': [0.0], 'GSM1943721': [0.0], 'GSM1943722': [0.0], 'GSM1943723': [1.0], 'GSM1943724': [0.0], 'GSM1943725': [0.0], 'GSM1943726': [1.0], 'GSM1943727': [1.0], 'GSM1943728': [0.0], 'GSM1943729': [0.0], 'GSM1943730': [1.0], 'GSM1943731': [1.0], 'GSM1943732': [1.0], 'GSM1943733': [1.0], 'GSM1943734': [0.0], 'GSM1943735': [1.0], 'GSM1943736': [0.0], 'GSM1943737': [1.0], 'GSM1943738': [1.0], 'GSM1943739': [1.0], 'GSM1943740': [1.0], 'GSM1943741': [1.0], 'GSM1943742': [1.0], 'GSM1943743': [1.0], 'GSM1943744': [1.0], 'GSM1943745': [1.0]}
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE75132.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown are Affymetrix probe IDs, not human gene symbols
# These identifiers follow the Affymetrix format (e.g., "1007_s_at", "1053_at")
# These will need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that contain probe IDs and gene symbols
# From the preview, we can see that 'ID' contains probe IDs (matching the expression data index)
# and 'Gene Symbol' contains gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Use the get_gene_mapping function to extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the first few rows of the mapped gene expression data
print("Preview of gene expression data after mapping to gene symbols:")
print(gene_data.head())


Preview of gene expression data after mapping to gene symbols:
          GSM1943705  GSM1943706  GSM1943707  GSM1943708  GSM1943709  \
Gene                                                                   
A1BG        1.032389    0.986202    0.936563    1.013798    0.965226   
A1BG-AS1    1.076463    1.023866    0.976135    0.767677    1.106595   
A1CF        2.353623    2.309421    2.039288    1.753579    2.287533   
A2M         1.072777    1.911737    2.092174    4.958586    1.907385   
A2M-AS1     0.918707    1.081293    0.888401    1.709019    1.016094   

          GSM1943710  GSM1943711  GSM1943712  GSM1943713  GSM1943714  ...  \
Gene                                                                  ...   
A1BG        0.811767    1.080926    1.102330    1.068037    0.972866  ...   
A1BG-AS1    0.924322    0.739589    0.594443    0.710322    0.855746  ...   
A1CF        2.433992    1.931220    2.282252    2.156752    2.383711  ...   
A2M         2.698037    1.543511    6.906710   

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Load the clinical data that was processed earlier
clinical_df = pd.read_csv(out_clinical_data_file)
# Set index for proper linking
clinical_df = clinical_df.set_index(clinical_df.columns[0])
print("Clinical data shape:", clinical_df.shape)
print("Clinical data preview:")
print(clinical_df.head())
print("Clinical data columns:", clinical_df.columns.tolist())

# Get the actual trait column name from the clinical data
# Based on previous steps, we know we converted trait_row (row 1 of the original matrix) 
# to a binary value representing cervical cancer progression
actual_trait_col = clinical_df.index[0]  # This gets the name of the binary trait we created
print(f"Actual trait column: {actual_trait_col}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)
print("Linked data columns preview:", linked_data.columns[:10].tolist())

# 3. Handle missing values in the linked data
# Use the actual trait column instead of the variable name
linked_data = handle_missing_values(linked_data, actual_trait_col)

# Rename the trait column to match the expected trait name
linked_data = linked_data.rename(columns={actual_trait_col: trait})

# 4. Determine whether the trait and demographic features are severely biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains cervical samples with HPV-16 infection, some with progression to high-grade lesions (CIN3+)"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE75132.csv
Clinical data shape: (1, 40)
Clinical data preview:
            GSM1943706  GSM1943707  GSM1943708  GSM1943709  GSM1943710  \
GSM1943705                                                               
0.0                0.0         0.0         0.0         0.0         0.0   

            GSM1943711  GSM1943712  GSM1943713  GSM1943714  GSM1943715  ...  \
GSM1943705                                                              ...   
0.0                0.0         1.0         1.0         1.0         0.0  ...   

            GSM1943736  GSM1943737  GSM1943738  GSM1943739  GSM1943740  \
GSM1943705                                                               
0.0                0.0         1.0         1.0         1.0         1.0   

            GSM1943741  GSM1943742  GSM1943743  GSM1943744  GSM1943745  
GSM1943705                                                              
0.0                1.0   

For the feature 'Cervical_Cancer', the least common label is '0.0' with 20 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Cervical_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cervical_Cancer/GSE75132.csv
